# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846776040096                   -0.70    4.2
  2   -7.852290110593       -2.26       -1.53    1.0
  3   -7.852646083799       -3.45       -2.52    3.2
  4   -7.852646677868       -6.23       -3.37    2.2
  5   -7.852646685852       -8.10       -4.77    1.0
  6   -7.852646686723       -9.06       -5.19    3.8
  7   -7.852646686730      -11.19       -6.14    1.5
  8   -7.852646686730      -12.60       -7.43    2.5
  9   -7.852646686730   +  -15.05       -7.51    2.0
 10   -7.852646686730   +    -Inf       -8.72    2.2
 11   -7.852646686730   +  -15.05       -9.46    2.0
 12   -7.852646686730      -15.05      -10.59    2.5
 13   -7.852646686730   +  -15.05      -10.64    1.0
 14   -7.852646686730   +    -Inf      -11.21    2.2
 15   -7.852646686730   +  -14.75      -11.91    2.2
 16   -7.852646686730      -14.57      -13.07    2.8


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846894264762                   -0.70           4.8
  2   -7.852528499007       -2.25       -1.64   0.80    2.0
  3   -7.852635239768       -3.97       -2.74   0.80    1.0
  4   -7.852646538745       -4.95       -3.21   0.80    2.0
  5   -7.852646674180       -6.87       -4.03   0.80    1.0
  6   -7.852646686297       -7.92       -4.70   0.80    1.8
  7   -7.852646686710       -9.38       -5.50   0.80    2.0
  8   -7.852646686729      -10.72       -6.62   0.80    2.0
  9   -7.852646686730      -12.39       -7.76   0.80    2.5
 10   -7.852646686730   +  -14.75       -8.63   0.80    2.5
 11   -7.852646686730      -14.75       -8.44   0.80    2.8
 12   -7.852646686730      -15.05       -9.59   0.80    1.0
 13   -7.852646686730   +    -Inf      -10.47   0.80    1.5
 14   -7.852646686730   +  -15.05      -10.96   0.80    2.2
 15   -7.852646686730      -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.425568e+01     2.941390e+00
 * time: 0.4644641876220703
     1     1.255442e+00     1.964259e+00
 * time: 0.7063579559326172
     2    -1.422608e+00     2.240613e+00
 * time: 0.8402481079101562
     3    -3.773446e+00     1.982681e+00
 * time: 0.8814761638641357
     4    -5.003320e+00     1.840952e+00
 * time: 0.920619010925293
     5    -6.691110e+00     1.185614e+00
 * time: 0.9597311019897461
     6    -7.356761e+00     4.506051e-01
 * time: 0.9988811016082764
     7    -7.617681e+00     1.580695e-01
 * time: 1.0256671905517578
     8    -7.720773e+00     2.096028e-01
 * time: 1.052422046661377
     9    -7.769587e+00     7.644061e-02
 * time: 1.079329013824463
    10    -7.790805e+00     9.894087e-02
 * time: 1.1072630882263184
    11    -7.805600e+00     1.081378e-01
 * time: 1.1353609561920166
    12    -7.819902e+00     5.816362e-02
 * time: 1.1630079746246338
    13    -7.839035e+00     4.054660e-02
 * time: 1.1904761791229

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846752636392                   -0.70    4.8
  2   -7.852308863939       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686709                   -2.54
  2   -7.852646686730      -10.67       -5.95
  3   -7.852646686730   +  -14.75      -12.59


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.6988970388668492e-14
|ρ_newton - ρ_scfv| = 2.0275310841393318e-13
|ρ_newton - ρ_dm|   = 1.0173593797284858e-9
